In [51]:
import os
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
from sqlalchemy import (
    Column, Integer, String, Text, DateTime, func, 
    cast, literal, text, select, Computed, Index,
)
from sqlalchemy.dialects import postgresql
from sqlalchemy.dialects.postgresql import TSVECTOR, REGCONFIG
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import Session, sessionmaker

In [52]:
def render_query(query):
    """
    Takes a SQLAlchemy query object and returns the fully rendered SQL query
    with all the arguments (bind parameters) inserted.
    
    Args:
        query: SQLAlchemy query object to render.
        
    Returns:
        str: Fully rendered SQL query as a string.
    """
    return str(query.compile(dialect=postgresql.dialect(), compile_kwargs={"literal_binds": True}))

In [55]:
def run_query(query, render=True, scalars=True):
    query_string = render_query(query) if render else str(query)
    print("QUERY::: ", query_string)
    with Session(engine) as session:
        if scalars:
            res = session.scalars(query)
        else:
            res = session.execute(query)
        print("\nRESULT::: ", list(res))
        return res, query_string

# Base SQLAlchemy example

## Create a Normal Table

### Create a db Engine

In [4]:
import os
from sqlalchemy import create_engine

engine = create_engine(
    os.environ.get("DATABASE_URL"), 
    echo=False,
)
engine

Engine(postgresql+psycopg://admin:***@db:5432/db)

### Define a Table

In [5]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text, DateTime, func

Base = declarative_base()

class Listing(Base):
    __tablename__ = 'listings'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True)
    created_at = Column(DateTime, default=func.now())
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now())
    
    def __repr__(self):
        return f"<Listing(id='{self.id}' name='{self.name}')>"


Listing.__table__.drop(engine, checkfirst=True)
Base.metadata.create_all(engine)

### Add Some Records

In [6]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    session.add_all([
        Listing(name="home"), 
        Listing(name="away")
    ])
    session.commit()

### Show the db Records

In [7]:
%load_ext sql

In [8]:
%%sql $DB_URL
select * from listings;

2 rows affected.


id,name,created_at,updated_at
1,home,2024-10-03 23:16:02.143959,2024-10-03 23:16:02.143959
2,away,2024-10-03 23:16:02.143959,2024-10-03 23:16:02.143959


# JSONB Example

## Create Table with JSONB

### Define a Table

In [73]:
from sqlalchemy.orm import declarative_base
from sqlalchemy import Column, Integer, String, Text, DateTime, func
from sqlalchemy.dialects.postgresql import JSONB

Base = declarative_base()

class Listing(Base):
    __tablename__ = 'listings'
    
    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True)
    rooms = Column(JSONB)
    created_at = Column(DateTime, default=func.now())
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now())
    
    def __repr__(self):
        return f"<Listing(id='{self.id}' name='{self.name}')>"


Listing.__table__.drop(engine, checkfirst=True)
Base.metadata.create_all(engine)

### Add Some Records

In [74]:
from sqlalchemy.orm import Session
with Session(engine) as session:
    session.add_all([
        Listing(name="home", rooms={
            "kitchen": {"color": "blue"},
        }), 
        Listing(name="away", rooms={
            "bedroom": {"color": "yellow"},
            "kitchen": {"color": "yellow"},
        }), 
        Listing(name="far", rooms={
            "bedroom": {"color": "blue"},
            "basement": {"color": "green"},
        }), 
        Listing(name="shed", rooms={
            "bedroom": None,
            "basement": {"color": "green"},
        }), 
        Listing(name="city", rooms={
            "basement": {"color": "green"},
            "bedroom": {"color": "green"},
        }),
        Listing(name="country", rooms={
            "basement": {"color": "green"},
        }),
    ])
    session.commit()

### Show the db Records

In [75]:
%%sql $DB_URL
select id,name,rooms from listings;

6 rows affected.


id,name,rooms
1,home,{'kitchen': {'color': 'blue'}}
2,away,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}"
3,far,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}}"
4,shed,"{'bedroom': None, 'basement': {'color': 'green'}}"
5,city,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}}"
6,country,{'basement': {'color': 'green'}}


### Filter `bedroom.color == yellow`

In [76]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms['bedroom']["color"].astext == "yellow",
)
_, query_string = run_query(query)

QUERY:::  SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (((listings.rooms -> 'bedroom')) ->> 'color') = 'yellow'

RESULT:::  [<Listing(id='2' name='away')>]


In [79]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms.op("->")('bedroom').op("->>")('color') == "yellow",
)
_, query_string = run_query(query)

QUERY:::  SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE ((listings.rooms -> 'bedroom') ->> 'color') = 'yellow'

RESULT:::  [<Listing(id='2' name='away')>]


In [80]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (((listings.rooms -> 'bedroom')) ->> 'color') = 'yellow'

1 rows affected.


id,name,rooms,created_at,updated_at
2,away,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615


### Filter `bedroom.color == yellow`

In [81]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms['bedroom'] == {"color": "yellow"},
)
_ = run_query(query, render=False)

QUERY:::  SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE listings.rooms[:rooms_1] = :param_1

RESULT:::  [<Listing(id='2' name='away')>]


In [82]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at
FROM listings 
WHERE (listings.rooms -> 'bedroom'::TEXT) = '{"color": "yellow"}'::JSONB

1 rows affected.


id,name,rooms,created_at,updated_at
2,away,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615


### Filter listings with bedrooms

In [83]:
from sqlalchemy import select

query = select(Listing).filter(
    Listing.rooms.op('?')('bedroom'))
_ = run_query(query, render=True)

QUERY:::  SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE listings.rooms ? 'bedroom'

RESULT:::  [<Listing(id='2' name='away')>, <Listing(id='3' name='far')>, <Listing(id='4' name='shed')>, <Listing(id='5' name='city')>]


In [84]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE listings.rooms ? 'bedroom'

4 rows affected.


id,name,rooms,created_at,updated_at
2,away,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
3,far,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
4,shed,"{'bedroom': None, 'basement': {'color': 'green'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
5,city,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615


### Filter `bedroom is Not Null`

In [85]:
query = select(Listing).filter(
    func.jsonb_extract_path_text(Listing.rooms, 'bedroom').isnot(None)
)
_ = run_query(query)

QUERY:::  SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_extract_path_text(listings.rooms, 'bedroom') IS NOT NULL

RESULT:::  [<Listing(id='2' name='away')>, <Listing(id='3' name='far')>, <Listing(id='5' name='city')>]


In [86]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_extract_path_text(listings.rooms, 'bedroom') IS NOT NULL

3 rows affected.


id,name,rooms,created_at,updated_at
2,away,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
3,far,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
5,city,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615


### Filter `kitchen and bathroom has a color`

In [87]:
query = select(Listing).filter(
    Listing.rooms.op('->')('kitchen').op('?')('color'))

_ = run_query(query)

QUERY:::  SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (listings.rooms -> 'kitchen') ? 'color'

RESULT:::  [<Listing(id='1' name='home')>, <Listing(id='2' name='away')>]


In [88]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (listings.rooms -> 'kitchen') ? 'color'

2 rows affected.


id,name,rooms,created_at,updated_at
1,home,{'kitchen': {'color': 'blue'}},2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
2,away,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615


In [89]:
query = select(Listing).filter(
    Listing.rooms.op('->')('bathroom').op('?')('color'))

_ = run_query(query)

QUERY:::  SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (listings.rooms -> 'bathroom') ? 'color'

RESULT:::  []


In [90]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE (listings.rooms -> 'bathroom') ? 'color'

0 rows affected.


id,name,rooms,created_at,updated_at


### Find all rooms with a green color using JSONPath

In [91]:
query = select(Listing).where(
    func.jsonb_path_exists(
        Listing.rooms, text("""'$.* ? (@.color == "green")'::jsonpath""")))

_ = run_query(query)

QUERY:::  SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_path_exists(listings.rooms, '$.* ? (@.color == "green")'::jsonpath)

RESULT:::  [<Listing(id='3' name='far')>, <Listing(id='4' name='shed')>, <Listing(id='5' name='city')>, <Listing(id='6' name='country')>]


In [92]:
%%sql $DB_URL 
SELECT listings.id, listings.name, listings.rooms, listings.created_at, listings.updated_at 
FROM listings 
WHERE jsonb_path_exists(listings.rooms, '$.* ? (@.color == "green")'::jsonpath)

4 rows affected.


id,name,rooms,created_at,updated_at
3,far,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
4,shed,"{'bedroom': None, 'basement': {'color': 'green'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
5,city,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}}",2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615
6,country,{'basement': {'color': 'green'}},2024-10-03 23:59:46.315615,2024-10-03 23:59:46.315615


### Use JSONB operators in the select 

In [93]:
query = select(
    Listing.id, 
    Listing.rooms.op("->>")('bedroom'),
    Listing.rooms.op("->")('bedroom').op("->>")('color'),
)
_ = run_query(query, scalars=False)

QUERY:::  SELECT listings.id, listings.rooms ->> 'bedroom' AS anon_1, (listings.rooms -> 'bedroom') ->> 'color' AS anon_2 
FROM listings

RESULT:::  [(1, None, None), (2, '{"color": "yellow"}', 'yellow'), (3, '{"color": "blue"}', 'blue'), (4, None, None), (5, '{"color": "green"}', 'green'), (6, None, None)]


In [94]:
%%sql $DB_URL 
SELECT listings.id, listings.rooms ->> 'bedroom' AS anon_1, (listings.rooms -> 'bedroom') ->> 'color' AS anon_2 
FROM listings

6 rows affected.


id,anon_1,anon_2
1,None,None
2,"{""color"": ""yellow""}",yellow
3,"{""color"": ""blue""}",blue
4,None,None
5,"{""color"": ""green""}",green
6,None,None


In [33]:
# https://www.postgresql.org/docs/9.6/functions-json.html
# https://www.postgresql.org/docs/current/datatype-json.html

In [ ]:
# https://github.com/json-path/JsonPath

# Search Example

### Generate a "Vectorized" representation of the text

In [105]:
%%sql $DB_URL 
select to_tsvector(
    'my name is Joe and I live in Chicago. I have used Django and kubernetes');

1 rows affected.


to_tsvector
'chicago':9 'django':13 'joe':4 'kubernet':15 'live':7 'name':2 'use':12


### Do the search terms exist in the vector? 

In [108]:
%%sql $DB_URL 
select to_tsvector(
    'my name is Joe and I live in Chicago. I have used Django and kubernetes')
@@ to_tsquery('Chicago & Django');

1 rows affected.


?column?
True


In [113]:
%%sql $DB_URL 
select to_tsvector(
    'my name is Joe and I live in Chicago. I have used Django and kubernetes')
@@ to_tsquery('DC');

1 rows affected.


?column?
False


In [114]:
Base = declarative_base()
from sqlalchemy.types import TypeDecorator

class TSVector(TypeDecorator):
    impl = TSVECTOR
    cache_ok = True  

class Listing(Base):
    __tablename__ = 'listings'
    
    id = Column(Integer, primary_key=True,)
    name = Column(String, unique=True)
    description = Column(Text(), nullable=True)
    rooms = Column(JSONB)
    created_at = Column(DateTime, default=func.now())
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now())

    ts_vector = Column(
        TSVector(), 
        Computed(
            func.setweight(
                func.to_tsvector(
                    cast(literal("english"), type_=REGCONFIG), 
                    func.coalesce(name, '')), 'A' ).concat(
            func.setweight(
                func.to_tsvector(
                    cast(literal("english"), type_=REGCONFIG), 
                    func.coalesce(description, '')), "B" )),
            persisted=True))
    
    __table_args__ = (
        Index('ix_video___ts_vector__', ts_vector, postgresql_using='gin'),)
    
    def __repr__(self):
        return f"<Listing(id='{self.id}' name='{self.name}')>"



Listing.__table__.drop(engine, checkfirst=True)
Base.metadata.create_all(engine)

In [115]:
from sqlalchemy.orm import Session
with Session(engine) as session:
    session.add_all([
        Listing(
            name="home", 
            description="No place like home. A great place to live.",
            rooms={
            "kitchen": {"color": "blue"},
        }), 
        Listing(
            name="away", 
            description="A house that is very yellow and bright themed.",
            rooms={
            "bedroom": {"color": "yellow"},
            "kitchen": {"color": "yellow"},
        }), 
        Listing(
            name="far", 
            description="A large house in the middle of the countryside and has a barnhouse theme.",
            rooms={
            "bedroom": {"color": "blue"},
            "basement": {"color": "green"},
        }), 
        Listing(
            name="city", 
            description="Green 18th century old house with big garage",
            rooms={
            "basement": {"color": "green"},
            "bedroom": {"color": "green"},
        }),
        Listing(name="country", rooms={
            "basement": {"color": "green"},
        }),
    ])
    session.commit()

In [116]:
%%sql $DB_URL
select id, name, description, ts_vector from listings;

5 rows affected.


id,name,description,ts_vector
1,home,No place like home. A great place to live.,"'great':7B 'home':1A,5B 'like':4B 'live':10B 'place':3B,8B"
2,away,A house that is very yellow and bright themed.,'away':1A 'bright':9B 'hous':3B 'theme':10B 'yellow':7B
3,far,A large house in the middle of the countryside and has a barnhouse theme.,'barnhous':14B 'countrysid':10B 'far':1A 'hous':4B 'larg':3B 'middl':7B 'theme':15B
4,city,Green 18th century old house with big garage,'18th':3B 'big':8B 'centuri':4B 'citi':1A 'garag':9B 'green':2B 'hous':6B 'old':5B
5,country,None,'countri':1A


In [117]:
from sqlalchemy.orm import Session
search_query = 'house yellow'

query = select(Listing).filter(
    Listing.ts_vector.op('@@')(
        func.plainto_tsquery(cast(literal("english"), type_=REGCONFIG), search_query)
    )
)

_ = run_query(query)

QUERY:::  SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, listings.ts_vector 
FROM listings 
WHERE listings.ts_vector @@ plainto_tsquery(CAST('english' AS REGCONFIG), 'house yellow')

RESULT:::  [<Listing(id='2' name='away')>]


In [118]:
%%sql $DB_URL
SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, listings.ts_vector 
FROM listings 
WHERE listings.ts_vector @@ plainto_tsquery(CAST('english' AS REGCONFIG), 'house yellow')

1 rows affected.


id,name,description,rooms,created_at,updated_at,ts_vector
2,away,A house that is very yellow and bright themed.,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-04 00:13:05.782165,2024-10-04 00:13:05.782165,'away':1A 'bright':9B 'hous':3B 'theme':10B 'yellow':7B


In [119]:
from sqlalchemy.orm import Session
search_query = 'house | yellow | country'

query = select(Listing).filter(
    Listing.ts_vector.op('@@')(
        func.to_tsquery(cast(literal("english"), type_=REGCONFIG), search_query)
    )
)

_ = run_query(query)

QUERY:::  SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, listings.ts_vector 
FROM listings 
WHERE listings.ts_vector @@ to_tsquery(CAST('english' AS REGCONFIG), 'house | yellow | country')

RESULT:::  [<Listing(id='2' name='away')>, <Listing(id='3' name='far')>, <Listing(id='4' name='city')>, <Listing(id='5' name='country')>]


In [120]:
%%sql $DB_URL
SELECT listings.id, listings.name, listings.description, listings.rooms, listings.created_at, listings.updated_at, listings.ts_vector 
FROM listings 
WHERE listings.ts_vector @@ to_tsquery(CAST('english' AS REGCONFIG), 'house | yellow | country');

4 rows affected.


id,name,description,rooms,created_at,updated_at,ts_vector
2,away,A house that is very yellow and bright themed.,"{'bedroom': {'color': 'yellow'}, 'kitchen': {'color': 'yellow'}}",2024-10-04 00:13:05.782165,2024-10-04 00:13:05.782165,'away':1A 'bright':9B 'hous':3B 'theme':10B 'yellow':7B
3,far,A large house in the middle of the countryside and has a barnhouse theme.,"{'bedroom': {'color': 'blue'}, 'basement': {'color': 'green'}}",2024-10-04 00:13:05.782165,2024-10-04 00:13:05.782165,'barnhous':14B 'countrysid':10B 'far':1A 'hous':4B 'larg':3B 'middl':7B 'theme':15B
4,city,Green 18th century old house with big garage,"{'bedroom': {'color': 'green'}, 'basement': {'color': 'green'}}",2024-10-04 00:13:05.782165,2024-10-04 00:13:05.782165,'18th':3B 'big':8B 'centuri':4B 'citi':1A 'garag':9B 'green':2B 'hous':6B 'old':5B
5,country,None,{'basement': {'color': 'green'}},2024-10-04 00:13:05.782165,2024-10-04 00:13:05.782165,'countri':1A


In [ ]:
# https://www.postgresql.org/docs/current/textsearch-intro.html#TEXTSEARCH-DOCUMENT

In [ ]:
from sqlalchemy import func, select
from sqlalchemy.orm import Session
from contextlib import contextmanager
from dataclasses import dataclass

@dataclass
class Lock:
    locked: bool
    lock_key: int


@contextmanager
def advisory_lock(session: Session, lock_value: str, commit: bool = True):
    """
    Context manager that acquires a PostgreSQL advisory lock using a hash of the lock_value
    upon entering and releases it upon exiting, using SQLAlchemy's syntax.
    
    Args:
        session (Session): SQLAlchemy session to interact with the database.
        lock_value (str): The value (string) to hash and generate the lock key.
    
    Usage:
        with advisory_lock(session, "some_unique_value"):
            # Critical section where the lock is held
    """
    try:
        # Acquire the advisory lock using hashtext and pg_advisory_xact_lock in a single call
        lock_key_query = select(func.hashtext(lock_value))
        lock_key = session.execute(lock_key_query).scalar()
        
        lock = session.execute(
            select(func.pg_try_advisory_xact_lock(lock_key))
        ).first()[0]
        # Yield control back to the caller inside the context
        yield Lock(locked=not lock, lock_key=lock_key)
    finally:
        # The lock is automatically released when the transaction ends (no explicit release needed)
        if commit:
            session.commit()

In [ ]:
from sqlalchemy.orm import sessionmaker
import time

Session = sessionmaker(autocommit=False, autoflush=False, bind=engine)

lock_value = "some_unique_string_value"  # This string will be hashed to generate the lock key

with Session() as session:
    with advisory_lock(session, lock_value, commit=True) as lock:
        if lock.locked:
            print("Skip because locked")
        else:
            # Critical section where the advisory lock is held
            # Perform operations that need the lock
            session.execute(text('select 1'))
            time.sleep(30)
            print(lock.locked, lock.lock_key)
